In [ ]:
""" 
branch from analysis_mem2
Replacse analysis_mem2.ipynb (excepts note therin) and analysis_modelexpt_multsession_taskmodelscore_100620

[OBSOLETE] all latest plots are in analysis_TEMPLATE
"""


In [1]:
%load_ext autoreload
%autoreload 2

%cd ..

/data1/code/python/drawmonkey


In [2]:
from tools.utils import * 
from tools.plots import *
from tools.analy import *
from tools.calc import *
from tools.analyplot import *
from tools.preprocess import *
from tools.dayanalysis import *
from analysis.line2 import *
from analysis.modelexpt import *

from pythonlib.drawmodel.analysis import *
from pythonlib.tools.stroketools import *

from pythonlib.tools.datetools import getDateList
from analysis.line2 import PROBEDATfromFD
import seaborn as sns
import pandas as pd
from analysis.line2 import probedat2strokefeats, getStandardizedTime
from analysis.modelexpt import *
from analysis.probedatTaskmodel import *
from analysis.plotsTaskScore import *


NOTE: need to not overwrite strokes_all_task, because then the orders saved will stop being accurate. Modify


## LEARNING ACROSS BLOKKS (AND ACROSS DAYS)

## 3. Learning across days

In [27]:
SEPARATE_BY_BLOCK = True # whether should make seperate tiemcourse plotes fore ach block num.
animallist = ["Red"]
# exptlist = ["biasdir7_1", "biasdir3", "biasdir4", "biasdir7_2"]
exptlist = ["biasdir5", "biasdir6"]
for expt in exptlist:
    # expt = "biasdir4"

    #####################################################################3
    # valstoplot = ["hausdorff_positive", "modelscore", "frac_touched"]
    valstoplot = []
    if expt in ["biasdir5", "biasdir6", "biasdir7_1"]:
        valstoplot.append("first_stroke_vert_pos")
    elif expt in ["biasdir3", "biasdir4", "biasdir7_2"]:
        valstoplot.append("first_stroke_horiz_pos")

    print(valstoplot)


    #####################################################################
    # 1) Load data across days

    for animal in animallist:
        FD, MD = loadMultDataForExpt(expt, animal, whichdates="all", metadatonly=False)

        PROBEDAT = loadProbeDatWrapper(FD, MD, getnumstrokes=True)
        Probedat = ProbedatTaskmodel(PROBEDAT, MD)

        # saving dir
        SAVEDIR = f"{FD[0]['fd']['params']['figuredir_notebook']}/analysis_modelexpt_multsession/{expt}/multday_{animal}_{MD['sdate']}_to_{MD['edate']}/taskmodel"
        import os
        os.makedirs(SAVEDIR, exist_ok=True)
        print(f"saving at {SAVEDIR}")

        if MD["strokmodel_kind"] is not None:
            # ***************************************** COMPUTE MODEL SCORE OFFLIEN
            PROBEDAT, fig = probedatOfflineScore(PROBEDAT, filtdict={"random_task":[False]}, 
                                           ploton=True)
            fig.savefig(f"{SAVEDIR}/offlineScorePairplot.pdf")



        # ********************************************************
        # **************************** extract dataframe
        DF = Probedat.pandas()


        if "first_stroke_vert_pos" in valstoplot:
            # First touch position
            DF["first_stroke_vert_pos"] = assignStrokeToLabel(Probedat, "first_stroke_vert_pos")
        elif "first_stroke_horiz_pos" in valstoplot:
            DF["first_stroke_horiz_pos"] = assignStrokeToLabel(Probedat, "first_stroke_horiz_pos")
        else:
            assert all([v in ["hausdorff_positive", "modelscore", "frac_touched"] for v in valstoplot])


        # -- add blokk
        DFtmp = extractFeature(Probedat, feat="blokk")
        DF["blokk"] = DFtmp["blokk"]
        DF["blokk_across_days"] = DFtmp["blokk_across_days"]


        # ==== 
        valstoplot_good = []
        for v in valstoplot:
            if v in DF.columns:
                valstoplot_good.append(v)
        valstoplot = valstoplot_good


        # ********************************************************
        # *********************************** PLOTS
        # === GOOD PLOT, summarize task schedules.
        fig = sns.catplot(x="date", y="task_stagecategory", col = "random_task", row="taskgroup", hue="taskgroup", sharex=True,sharey=False,
                    data=DF, height=10, aspect=2, orient="v")
        fig.savefig(f"{SAVEDIR}/overview1.pdf")

        # ==== GOOD PLOT (scatter and means)
        for y in valstoplot:
            ALPHA=0.4
            if y in ["modelcomp", "modelcompflipped"]:
                YLIM=(-0.7, 0.7)
            elif y=="modelcomp_offline":
                YLIM=(-1,1)
            elif y=="hausdorff_positive":
                YLIM=np.percentile(DF["hausdorff_positive"], q=[0.5, 99.5])
            elif y=="nstrokesactual":
                YLIM=np.percentile(DF["nstrokesactual"], q=[0.5, 99.5])
                YLIM = [YLIM[0]-0.5, YLIM[1]+0.5]
                ALPHA=0.15
            else:
                YLIM = None

            # === 1) All Timecourses
            g = sns.FacetGrid(DF, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
            g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
            g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                          "alpha":ALPHA,
                                                                  "s":40, 
                                                                  "palette":{1:"r", 2:"b"}})
            g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
            g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1.pdf")

            # == one plot for each block
            blocks = set(DF["block"])
            if SEPARATE_BY_BLOCK:
                for b in blocks:
                    DFblock = DF[DF["block"]==b]
                    g = sns.FacetGrid(DFblock, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
                    g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
                    g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                                  "alpha":ALPHA,
                                                                          "s":40, 
                                                                          "palette":{1:"r", 2:"b"}})
                    g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                    g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1-block{b}.pdf")
                plt.close("all")

            # === summary (epoch1 vs. epoch2)
            DFsummary = DF[DF["keepforsummary"]==True]
            if len(DFsummary)>0:
                g = sns.FacetGrid(DFsummary, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                  sharex=False, sharey=True, ylim=YLIM)
                g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                      "s":4, 
                                                                      "palette":{1:"r", 2:"b"}})
                g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary.pdf")

                if SEPARATE_BY_BLOCK:
                    for b in blocks:
                        DFsummaryblock = DFsummary[DFsummary["block"]==b]
                        g = sns.FacetGrid(DFsummaryblock, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                          sharex=False, sharey=True, ylim=YLIM)
                        g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                              "s":4, 
                                                                              "palette":{1:"r", 2:"b"}})
                        g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                        g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                        g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary-block{b}.pdf")

                    plt.close("all")



        # ***************************************
        # ***************************************
        # [one plot for each unqiue task]
        plotEachUniqueTask(DF, PROBEDAT, valstoplot, SAVEDIR)

        plotEachUniqueTaskGrouped(DF, valstoplot, SAVEDIR)

        # ********************************************************
        # *********************************** PLOTS
        # === GOOD PLOT, summarize task schedules.
        fig = sns.catplot(x="date", y="task_stagecategory", col = "random_task", row="taskgroup", hue="taskgroup", sharex=True,sharey=False,
                    data=DF, height=10, aspect=2, orient="v")
        fig.savefig(f"{SAVEDIR}/overview1.pdf")

        # ==== GOOD PLOT (scatter and means)
        for y in valstoplot:
            ALPHA=0.4
            if y in ["modelcomp", "modelcompflipped"]:
                YLIM=(-0.7, 0.7)
            elif y=="modelcomp_offline":
                YLIM=(-1,1)
            elif y=="hausdorff_positive":
                YLIM=np.percentile(DF["hausdorff_positive"], q=[0.5, 99.5])
            elif y=="nstrokesactual":
                YLIM=np.percentile(DF["nstrokesactual"], q=[0.5, 99.5])
                YLIM = [YLIM[0]-0.5, YLIM[1]+0.5]
                ALPHA=0.15
            else:
                YLIM = None

            # === 1) All Timecourses
            g = sns.FacetGrid(DF, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
            g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
            g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                          "alpha":ALPHA,
                                                                  "s":40, 
                                                                  "palette":{1:"r", 2:"b"}})
            g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
            g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1.pdf")

            # == one plot for each block
            blocks = set(DF["block"])
            if SEPARATE_BY_BLOCK:
                for b in blocks:
                    DFblock = DF[DF["block"]==b]
                    g = sns.FacetGrid(DFblock, row="task_stagecategory", col="taskgroup", height=4, aspect=2, 
                              sharex=True, sharey=True, ylim=YLIM)
                    g.map(sns.lineplot, "tvalday", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":2})
                    g.map(sns.scatterplot, "tval", y, "epoch", **{"marker":"x", 
                                                                  "alpha":ALPHA,
                                                                          "s":40, 
                                                                          "palette":{1:"r", 2:"b"}})
                    g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                    g.savefig(f"{SAVEDIR}/alltimecourses-{y}-1-block{b}.pdf")
                plt.close("all")

            # === summary (epoch1 vs. epoch2)
            DFsummary = DF[DF["keepforsummary"]==True]
            if len(DFsummary)>0:
                g = sns.FacetGrid(DFsummary, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                  sharex=False, sharey=True, ylim=YLIM)
                g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                      "s":4, 
                                                                      "palette":{1:"r", 2:"b"}})
                g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary.pdf")

                if SEPARATE_BY_BLOCK:
                    for b in blocks:
                        DFsummaryblock = DFsummary[DFsummary["block"]==b]
                        g = sns.FacetGrid(DFsummaryblock, row="task_stagecategory", col="taskgroup", height=4, aspect=1.2, 
                                          sharex=False, sharey=True, ylim=YLIM)
                        g.map(sns.swarmplot, "epoch", y, "epoch", **{"alpha":ALPHA,
                                                                              "s":4, 
                                                                              "palette":{1:"r", 2:"b"}})
                        g.map(sns.pointplot, "epoch", y, **{"err_style":"bars", "ci":68, "color":"k", "linewidth":1})
                        g.map(plt.axhline, **{"color":[0.7, 0.7, 0.7]})
                        g.savefig(f"{SAVEDIR}/alltimecourses-{y}-summary-block{b}.pdf")

                    plt.close("all")



        # ***************************************
        # ***************************************
        # [one plot for each unqiue task]
        plotEachUniqueTask(DF, PROBEDAT, valstoplot, SAVEDIR)
        plotEachUniqueTaskGrouped(DF, valstoplot, SAVEDIR)







['first_stroke_vert_pos']
This is metadat:
{'sdate': 210111, 'edate': 210112, 'strokmodel_kind': None, 'strokmodel_tstamp': None, 'datecategories': {'210111': 1, '210112': 1}, 'dates_for_summary': ['210111', '210112'], 'matchedstrokes': None, 'exptnames': ['biasdir5'], 'tasknames_G1': [], 'tasknames_G2': ['linePlusLv2', '3line'], 'tasknames_G3': ['2linePlusL', 'LplusL', '3linePlusL', '4line'], 'description': None, 'expt': 'biasdir5', 'task_train_test': {'probe1_liketrain': 'train', 'probe1_nostrokeconstraint': 'train', 'probe2_liketrain': 'train', 'probe2_nostrokeconstraint': 'train', 'probe3_hdpos': 'test', 'probe1': 'train', 'probe2': 'train', 'probe3': 'test', 'probe4': 'test', 'train': 'train'}}

Getting these dates:
['210111', '210112']
Red 210111 biasdir5 0
- No h5 file for Red, 210111, biasdir5, 0 - returning None!
Red 210111 biasdir5 1
- No h5 file for Red, 210111, biasdir5, 1 - returning None!
Red 210111 biasdir5 2
-- loaded presaved data: /data2/animals/Red/210111/210111_1457

/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the swarmplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:650: UserWarning: Using the swarmplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/categorical.py:1296: UserWarning: 69.3% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the pointplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.p

/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the pointplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the swarmplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:650: UserWarning: Using the swarmplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the pointplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Us

{'first_stroke_vert_pos': ['mean'], 'task_stagecategory': 'first'}
{'first_stroke_vert_pos': ['mean'], 'task_stagecategory': 'first'}


/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the swarmplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:650: UserWarning: Using the swarmplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/categorical.py:1296: UserWarning: 69.3% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the pointplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.p

...

/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the swarmplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:650: UserWarning: Using the swarmplot function without specifying `hue_order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the pointplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the swarmplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:650: UserWarning: Us

/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the pointplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:411: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=figsize)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:645: UserWarning: Using the swarmplot function without specifying `order` is likely to produce an incorrect plot.
  warnings.warn(warning)
/home/lucast4/miniconda3/envs/drag2/lib/python3.8/site-packages/seaborn/axisgrid.py:650: UserWarning: Using the swarmplot function without specifying `hue_order

{'first_stroke_vert_pos': ['mean'], 'task_stagecategory': 'first'}
{'first_stroke_vert_pos': ['mean'], 'task_stagecategory': 'first'}
This is metadat:
{'sdate': 210113, 'edate': 210113, 'strokmodel_kind': None, 'strokmodel_tstamp': None, 'datecategories': {'210113': 1}, 'dates_for_summary': ['210113'], 'matchedstrokes': None, 'exptnames': ['biasdir6'], 'tasknames_G1': [], 'tasknames_G2': ['linePlusLv2', '3line'], 'tasknames_G3': ['2linePlusL', 'LplusL', '3linePlusL', '4line'], 'description': None, 'expt': 'biasdir6', 'task_train_test': {'probe1_liketrain': 'train', 'probe1_nostrokeconstraint': 'train', 'probe2_liketrain': 'train', 'probe2_nostrokeconstraint': 'train', 'probe3_hdpos': 'test', 'probe1': 'train', 'probe2': 'train', 'probe3': 'test', 'probe4': 'test', 'train': 'train'}}

Getting these dates:
['210113']
Red 210113 biasdir6 0
- No h5 file for Red, 210113, biasdir6, 0 - returning None!
Red 210113 biasdir6 1
-- loaded presaved data: /data2/animals/Red/210113/210113_144213_bias

-- date: 210113
found this trial(day): 57, assigned idx: 0
found this trial(day): 156, assigned idx: 1
found this trial(day): 350, assigned idx: 2
-- date: 210113
found this trial(day): 57, assigned idx: 0
found this trial(day): 156, assigned idx: 1
found this trial(day): 350, assigned idx: 2
-- date: 210113
found this trial(day): 23, assigned idx: 0
found this trial(day): 138, assigned idx: 1
found this trial(day): 385, assigned idx: 2
-- date: 210113
found this trial(day): 23, assigned idx: 0
found this trial(day): 138, assigned idx: 1
found this trial(day): 385, assigned idx: 2
-- date: 210113
found this trial(day): 175, assigned idx: 0
found this trial(day): 334, assigned idx: 1
-- date: 210113
found this trial(day): 175, assigned idx: 0
found this trial(day): 334, assigned idx: 1
-- date: 210113
found this trial(day): 8, assigned idx: 0
found this trial(day): 153, assigned idx: 1
found this trial(day): 336, assigned idx: 2
-- date: 210113
found this trial(day): 8, assigned idx: 0


-- date: 210113
found this trial(day): 11, assigned idx: 0
found this trial(day): 198, assigned idx: 1
found this trial(day): 358, assigned idx: 2
-- date: 210113
found this trial(day): 49, assigned idx: 0
found this trial(day): 135, assigned idx: 1
found this trial(day): 372, assigned idx: 2
-- date: 210113
found this trial(day): 6, assigned idx: 0
found this trial(day): 182, assigned idx: 1
found this trial(day): 380, assigned idx: 2
-- date: 210113
found this trial(day): 5, assigned idx: 0
found this trial(day): 181, assigned idx: 1
found this trial(day): 343, assigned idx: 2
-- date: 210113
found this trial(day): 37, assigned idx: 0
found this trial(day): 197, assigned idx: 1
found this trial(day): 382, assigned idx: 2
-- date: 210113
found this trial(day): 159, assigned idx: 0
-- date: 210113
found this trial(day): 47, assigned idx: 0
found this trial(day): 194, assigned idx: 1
found this trial(day): 331, assigned idx: 2
-- date: 210113
found this trial(day): 35, assigned idx: 0
f

## More plots, Plot task behaviors (below not cleaned up, but works)
### LOTS OF PLOTS (ALL TRIALS, EXAMPLE TRIALS, ETC)

In [9]:
# expt = "biasdir4"
# thingstoplot = ["alltrials", "egtrials"]
thingstoplot = ["alltrials", "egtrials", "egtrials_byblock"]

# expt = "lines5"
# # max_strokenum = 2 # for single stroke plots
# thingstoplot = ["alltrials", "egtrials"]
# # thingstoplot = ["egtrials"]

In [10]:
def plotalltrials(PROBEDATthis, fdlist, stage, tasklist, SAVEDIR, datelist=None, blocklist=None, ver="bydate"):
    tasklist = sorted(set([P["unique_task_name"] for P in PROBEDATthis if P["task_stagecategory"]==stage]))
    if len(tasklist)>100:
        assert False, "why so many tasks for this stage?"

    # -- Plot 2d grid, separated by days
    if ver=="bydate":
        assert datelist is not None
        CAT1 = ["date", datelist]
    elif ver=="byblock":
        assert blocklist is not None
        CAT1 = ["block", blocklist]
    else:
        print(ver)
        assert False, "not coded"
    CAT2 = ["unique_task_name", tasklist]
    fdlist = None

    # == 1) All behavior trials, all strokes overlaid
    plotargs = {"zoom":True, "plotver":"order", "markersize":4, "alpha":0.25}
    # fdlist = [P["filedata"][0]() for P in PROBEDATthis]
    # fdlist = [FD[P["ii"]]["fd"] for P in PROBEDATthis]
    
    fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="beh", plotargs=plotargs);
    fig.savefig(f"{SAVEDIR}/datebycategory_beh-{stage}-{ver}.pdf")
    fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="task", plotargs=plotargs);
    fig.savefig(f"{SAVEDIR}/datebycategory_task-{stage}-{ver}.pdf")

    # == 2) Split by stroke number (one plot for each stroke)
    if False:
        plotargs = {"zoom":True, "plotver":"order", "markersize":3, "alpha":0.2}
        strokenums_to_plot_alone=list(range(max_strokenum+1))
        overlay_stroke_mean=False

        fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="behtask", 
                              plotargs=plotargs, strokenums_to_plot_alone=strokenums_to_plot_alone, 
                             overlay_stroke_mean = overlay_stroke_mean);
        fig.savefig(f"{SAVEDIR}/alltrials_datebycategory_eachstroke_beh_{stage}.pdf")


    # == 3) All strokes (faint) and overlay average
    if False:
        plotargs = {"zoom":True, "plotver":"order", "markersize":2, "alpha":0.15}

        overlay_stroke_mean=True
        fig = plotTrial2dGrid(PROBEDATthis, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver="beh", 
                              plotargs=plotargs, overlay_stroke_mean=overlay_stroke_mean);
        fig.savefig(f"{SAVEDIR}/alltrials_datebycategory_strokemeans_{stage}.pdf")


In [11]:
debug=False

In [12]:
def plotBehaviors(expt, animal, thingstoplot, debug):
    FD, MD = loadMultDataForExpt(expt, animal, whichdates="all", metadatonly=False)
    PROBEDAT = loadProbeDatWrapper(FD, MD, getnumstrokes=True)

    # get all fixed tasks of a particular kind
    kindlist = set([P["kind"] for P in PROBEDAT if P["random_task"]==False])
    task_per_kind = {}
    for kind in kindlist:
        tasklist = set([P["unique_task_name"] for P in PROBEDAT if P["kind"]==kind])
        task_per_kind[kind]=sorted(tasklist)

    print("tasks per kind found")
    for k, v in task_per_kind.items():
        print("----")
        print(f"=={k}")
        [print(vv) for vv in v]

    ## PLOT - all trials, 2d grid sorted by date and task category
    # only keep data for fixed tasks.
    PROBEDATthis = [P for P in PROBEDAT if P["random_task"]==False]
    datelist = sorted(set([P["date"] for P in PROBEDATthis]))
    blocklist = sorted(set([P["block"] for P in PROBEDATthis]))
    fdlist = None
    
    # for each stage, make a 2d grid plot (date x task)
    if "alltrials" in thingstoplot:
        # saving dir
        sdate = MD["sdate"]
        edate = MD["edate"]
        SAVEDIR = f"{FD[0]['fd']['params']['figuredir_notebook']}/analysis_modelexpt_multsession/{expt}/multday_{animal}_{sdate}_to_{edate}/alltrials"
        import os
        os.makedirs(SAVEDIR, exist_ok=True)
        print(f"saving at {SAVEDIR}")
    
        # all categories that have fixed tasks
        stagelist = set([P["task_stagecategory"] for P in PROBEDATthis if P["random_task"]==False])
        for stage in stagelist:
            plotalltrials(PROBEDATthis, fdlist, stage, tasklist, SAVEDIR, datelist=datelist, ver="bydate")
            plotalltrials(PROBEDATthis, fdlist, stage, tasklist, SAVEDIR, blocklist=blocklist, ver="byblock")
        

    # ==== PLOT ALL TRIALS
    if "egtrials" in thingstoplot:
        
        # saving dir
        sdate = MD["sdate"]
        edate = MD["edate"]
        SAVEDIR = f"{FD[0]['fd']['params']['figuredir_notebook']}/analysis_modelexpt_multsession/{expt}/multday_{animal}_{sdate}_to_{edate}/egtrials"
        import os
        os.makedirs(SAVEDIR, exist_ok=True)
        print(f"saving at {SAVEDIR}")

        if debug:
            # making plots for lab meeting..
            tasklist = set([P["unique_task_name"] for P in PROBEDATthis if P["random_task"]==False
               and (P["task_stagecategory"] in ["LplusL", "2linePlusL", "3linePlusL"] or "linePlusLv2_51" in P["unique_task_name"])])
        else:
            tasklist = set([P["unique_task_name"] for P in PROBEDATthis if P["random_task"]==False])
        NMAX = 20 # trials to plot, starting from 1st trial int he day


        for task in tasklist:

            PD = [P for P in PROBEDATthis if P["random_task"]==False and P["unique_task_name"]==task]
            
            # only do reverse if more trials than will plot. otherwise is redundant.
            PD, countlist = probeDatIndexWithinDay(PD, task, reverse_order=False)
            if max(countlist)>NMAX:
                rev = [False, True]
            else:
                rev= [False]
            
            for reverse in rev:
                # -- task presentation num as column
                for P in PD:
                    P["idx_today_uniquetask"] = None
                PD, countlist = probeDatIndexWithinDay(PD, task, reverse_order=reverse);

                # -- how many examples to plot?
                ntoplot = min((max(countlist), NMAX))

                # -- Plot 2d grid, separated by days
                CAT1 = ["date", datelist]
                CAT2 = ["idx_today_uniquetask", range(ntoplot)]
                fdlist = None

                # == 1) All behavior trials, all strokes overlaid
                plotargs = {"zoom":True, "plotver":"order", "markersize":8, "alpha":0.7}
                plot_task_last_col = True
                ver = "beh"

                fig = plotTrial2dGrid(PD, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver=ver, 
                                      plotargs=plotargs, plot_task_last_col=plot_task_last_col);
                if reverse:
                    fig.savefig(f"{SAVEDIR}/datebyexample_revchronorder_{task}.pdf")
                else:
                    fig.savefig(f"{SAVEDIR}/datebyexample_chronorder_{task}.pdf")

        plt.close("all")

    # ==== PLOT ALL TRIALS
    if "egtrials_byblock" in thingstoplot:
        tasklist = set([P["unique_task_name"] for P in PROBEDATthis if P["random_task"]==False])
        datelist = set([P["date"] for P in PROBEDATthis if P["random_task"]==False])
        NMAX = 20 # trials to plot, starting from 1st trial int he day
        reverse = False

        # saving dir
        sdate = MD["sdate"]
        edate = MD["edate"]
        SAVEDIR = f"{FD[0]['fd']['params']['figuredir_notebook']}/analysis_modelexpt_multsession/{expt}/multday_{animal}_{sdate}_to_{edate}/egtrials_byblock"
        import os
        os.makedirs(SAVEDIR, exist_ok=True)
        print(f"saving at {SAVEDIR}")

#         OLD VERSION - one plot per day.
#         for task in tasklist:
#             for date in datelist:
#                 PD = [P for P in PROBEDATthis if P["random_task"]==False and P["unique_task_name"]==task
#                      and P["date"]==date]
                
#                 # -- task presentation num as column
#                 for P in PD:
#                     P["idx_today_uniquetask"] = None
#                 PD, countlist = probeDatIndexWithinDay(PD, task, reverse_order=reverse);

#                 # -- new column, which is date-block as string
#                 for P in PD:
#                     P["date-block"] = [f"{P['date']}-{P['block']}" for P in PD]
                
#                 # -- how many examples to plot?
#                 ntoplot = min((max(countlist), NMAX))

#                 # -- Plot 2d grid, separated by days
#                 blocklist = sorted(set([P["block"] for P in PD]))
#                 CAT1 = ["block", blocklist]
#                 CAT2 = ["idx_today_uniquetask", range(ntoplot)]
#                 fdlist = None

#                 # == 1) All behavior trials, all strokes overlaid
#                 plotargs = {"zoom":True, "plotver":"order", "markersize":8, "alpha":0.7}
#                 plot_task_last_col = True
#                 ver = "beh"

#                 fig = plotTrial2dGrid(PD, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver=ver, 
#                                       plotargs=plotargs, plot_task_last_col=plot_task_last_col);
#                 fig.savefig(f"{SAVEDIR}/egtrials_datebyexample_chronorder-{task}-{date}.pdf")
#         assert False

#         plt.close("all")

        for task in tasklist:
            PD = [P for P in PROBEDATthis if P["random_task"]==False and P["unique_task_name"]==task]

            # -- task presentation num as column
            for P in PD:
                P["idx_today_uniquetask"] = None
            PD, countlist = probeDatIndexWithinDay(PD, task, reverse_order=reverse);

            # -- new column, which is date-block as string
            for P in PD:
                P["date_block"] = f"{P['date']}-{P['block']}"
            date_block_list = sorted(set([P["date_block"] for P in PD]))

            # -- how many examples to plot?
            ntoplot = min((max(countlist), NMAX))

            # -- Plot 2d grid, separated by days
            blocklist = sorted(set([P["block"] for P in PD]))
            CAT1 = ["date_block", date_block_list]
            CAT2 = ["idx_today_uniquetask", range(ntoplot)]
            fdlist = None

            # == 1) All behavior trials, all strokes overlaid
            plotargs = {"zoom":True, "plotver":"order", "markersize":8, "alpha":0.7}
            plot_task_last_col = True
            ver = "beh"

            fig = plotTrial2dGrid(PD, fdlist = fdlist, cat1 = CAT1, cat2 = CAT2, ver=ver, 
                                  plotargs=plotargs, plot_task_last_col=plot_task_last_col, 
                                 clean=True);
            fig.savefig(f"{SAVEDIR}/datebyexample_chronorder-{task}.pdf")

            plt.close("all")    

In [ ]:

# Plot same task across days

# 1) Load data across days
from pythonlib.tools.datetools import getDateList
# expt = "lines2"
for animal in ["Pancho", "Red"]:
    plotBehaviors(expt, animal)


## Plot all tasks (no behavior)


## IN PROGRESS - not sure if this is good

In [ ]:
if False:
        # ************** PLOT ALL FIXED TASKS (collect across days)
    for TRAINTEST in ["train", "test"]:

        # then plot all
        tasknames = sorted(set([P["unique_task_name"] for P in PROBEDAT
                          if P["random_task"]==False
                                and task_train_test[P["kind"]]==TRAINTEST]))

        # get one index per taskname
        fdlist = []
        triallist =[]
        titlelist = []
        for task in tasknames:
            fdlist.append([P["filedata"] for P in PROBEDAT
                            if P["unique_task_name"]==task][0])
            triallist.append([P["trial"] for P in PROBEDAT
                            if P["unique_task_name"]==task][0])
            titlelist.append(task)
        print(titlelist)

        fig = plotMultTrialsSimple(fdlist, trials_list=triallist, rand_subset=None, 
                                   empty_title=False, zoom=True, plot_fix=True, alpha=1, titles=titlelist,
                            plotargs={"plot_task_stimulus":True, 
                                    "plot_drawing_behavior":False, "nakedplot":True,
                                   })
        fig.savefig(f"{SAVEDIR}/alltasks_fixed_alldays_{TRAINTEST}.pdf")



    # *** FOR EACH DAY, PLOT SUBSET OF RANDOM TASKS.
    TRAINTEST = "train"
    for date in DATELIST:
        print(f"======= {date}")
        traintaskcats = sorted(set([P["task_stagecategory"] for P in PROBEDAT 
                                    if P["date"]==date
                                   and P["random_task"]==True
                                   and task_train_test[P["kind"]]==TRAINTEST]))
        for cat in traintaskcats:
            print(f"--- {cat}")

            # then just plot example tasks
            fdlist = [P["filedata"] for P in PROBEDAT
                              if P["date"]==date
                              and P["random_task"]==True
                            and P["task_stagecategory"]==cat
                     and task_train_test[P["kind"]]==TRAINTEST]
            triallist = [P["trial"] for P in PROBEDAT
                              if P["date"]==date
                              and P["random_task"]==True
                            and P["task_stagecategory"]==cat
                     and task_train_test[P["kind"]]==TRAINTEST]
            titlelist = [P["unique_task_name"] for P in PROBEDAT
                              if P["date"]==date
                              and P["random_task"]==True
                            and P["task_stagecategory"]==cat
                     and task_train_test[P["kind"]]==TRAINTEST]

            Nrand = 60
            fig = plotMultTrialsSimple(fdlist, trials_list=triallist, rand_subset=Nrand, 
                                       empty_title=False, zoom=True, plot_fix=True, alpha=1, titles=titlelist,
                                plotargs={"plot_task_stimulus":True, 
                                        "plot_drawing_behavior":False, "nakedplot":True,
                                       })
            fig.savefig(f"{SAVEDIR}/alltasks-random-{date}-{cat}-{TRAINTEST}.pdf")
            plt.close()


## BELOW - OLD SCRATCH

## SCRATCH